# Predicting structural breaks in sequential music data

Uses sequential data + regression to model the time series, then change point detection on top in order to detect structural breaks.

This version is only intended to handle one musical piece (+ its tagging information) at a time, so no concatenated Rachmaninoff pieces for instance (unless you have tagged the entire stream).

In [23]:
import os

# name for analysis running
ANALYSIS_NAME = "core1"

""" 
YOUR PARAMETERS - CHANGE THESE!

STEP1:
Put your MIDI files into the folder final_datasets/

STEP2:
Make sure your MIDI files have standardized naming, specifically, have
a common substring in the filename <MUSIC_FILE_REGEX>.
*If you include structural break tagging, add your tagging MIDI file
as <original_filename>_TAGGED_<time_series_name>.mid.
Example for MUSIC_FILE_REGEX="kapustin_variations":
    - kapustin_variations.mid:
        original MIDI file
    - kapustin_variations.wav:
        WAV file to accompany original MIDI
    - kapustin_variations_TAGGED_pitch_mean.mid:
        structural break tagging information for kapustin_variations.mid
    - kapustin_variations_TAGGED_pitch_mean.wav:
        WAV file to accompany tagging information
        (= original + tagging audio)

STEP3:
Change parameters here. Again, note that the string <MUSIC_FILE_REGEX>
will be used to filter for all MIDI files to analyze.
- corpus: name for corpus (only for output naming)
- music file regex: for searching pieces in final_datasets
- time series variable: time series to analyze (see available below)
"""

CORPUS = "saarland"
# IMPORTANT: should at most match 1 MIDI file in final_datasets/
MUSIC_FILE_REGEX = "Mozart_KV265_006"
TIME_SERIES_VARIABLE = "pitch_mean"

### other metadata
PIECE_NAME = "{}".format(MUSIC_FILE_REGEX)
PIANIST = "Saarland"
DATE = "May 1, 2021"

""""""

ANALYSIS_KEY = '__'.join([ANALYSIS_NAME, CORPUS, MUSIC_FILE_REGEX, TIME_SERIES_VARIABLE])
TIME_SERIES_VARIABLE_FMT = TIME_SERIES_VARIABLE.replace("_", " ").title()

### All structural break tagging MIDI fileswill have this in the filename
### For instance, "kapustin_variations_BEAT.midi" will be a structural break tagging file
### for the core music file "kapustin_variations.midi"
STRUCTURAL_BREAK_REGEX = "TAGGED"

### Use only the first filename that shows up. This notebook is only
### designed to find and run on 1 piece.
USE_ONLY_FIRST_MIDI_STREAM = True

### Validate time series variable:
### Should be one of the following
time_series_variables = ['n_notes',
 'pitch_min', 'pitch_mean', 'pitch_median', 'pitch_max',
 'velocity_min', 'velocity_mean', 'velocity_median', 'velocity_max',
 'duration_min', 'duration_mean', 'duration_median', 'duration_max']
if TIME_SERIES_VARIABLE not in time_series_variables:
    raise Exception("Time series variable not found.")

### Set up overall folder for task 1
### Analysis will use and concatenate all the MIDI tracks of interest in this folder,
### where tracks of interest have MIDI_STREAM_REGEX in filename.
INPUT_FOLDER = "final_datasets"
MIDI_STREAM_REGEX = MUSIC_FILE_REGEX
add_input_folder = lambda x: "{}/{}".format(INPUT_FOLDER, x).replace("//", "/")
print(INPUT_FOLDER)

### Output will be stored in folder (and subfolders of):
### results/<ANALYSIS_NAME>/<CORPUS>/<MUSIC_FILE_REGEX>/<TIME_SERIES_VARIABLE>
OUTPUT_FOLDER = "results"
if not os.path.exists(OUTPUT_FOLDER):
    os.mkdir(OUTPUT_FOLDER)
for v in [ANALYSIS_NAME, CORPUS, MUSIC_FILE_REGEX, TIME_SERIES_VARIABLE]:
    OUTPUT_FOLDER = "{}/{}".format(OUTPUT_FOLDER, v)
    if not os.path.exists(OUTPUT_FOLDER):
        os.mkdir(OUTPUT_FOLDER)
add_output_path = lambda x: "{}/{}".format(OUTPUT_FOLDER, x)
print(OUTPUT_FOLDER)

final_datasets
results/core1/saarland/Mozart_KV265_006/pitch_mean


In [24]:
from __future__ import division
from more_itertools import peekable
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as scp
import pickle
import magenta
import os, time, re, json, glob
%matplotlib inline
from IPython.core.display import display, HTML
### change width of notebook display
# display(HTML("<style>.container { width:70% !important; }</style>"))
from pathlib import Path
import ipdb;
def debug(): ipdb.set_trace() # debugging starts here
pd.set_option('display.float_format', lambda x: '%.4f' % x)
np.set_printoptions(suppress=True)
np.random.seed(42)

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# for exposing API
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from flask import Flask
from flask_restful import Resource, Api

JUPYTER_PICKLE_FILE = "config/shared_jupyter_data.pkl"
def write_shared_jupyter(key, value, path=JUPYTER_PICKLE_FILE, overwrite=False):
    if (os.path.exists(path)):
        with open(path, "rb") as fp:
            shared_jupyter_data = pickle.load(fp)
        if overwrite:
            shared_jupyter_data = {key: value}
        else:
            shared_jupyter_data[key] = value
    else:
        shared_jupyter_data = {key: value}
    with open(path, 'wb') as fp: 
        pickle.dump(shared_jupyter_data, fp)

def read_shared_jupyter(key=None, path=JUPYTER_PICKLE_FILE):
    if (os.path.exists(path)):
        with open(path, "rb") as fp:
            shared_jupyter_data = pickle.load(fp)
            if key is not None:
                if key in shared_jupyter_data:
                    return(shared_jupyter_data[key])
                else:
                    print("Not found!")
                    return(None)
            else:
                return(shared_jupyter_data)
    else:
        print("No data")

def pandasToJson(df):
    return(df.to_json(orient="split"))
def jsonToPandas(json):
    return(pd.read_json(json, orient="split"))

pd.set_option('display.float_format', lambda x: "%.3f" % x)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

FIG_WIDTH = 1600
FIG_HEIGHT = 800

PITCH_MIN = 20
PITCH_MAX = 120
VELOCITY_MIN = 0
VELOCITY_MAX = 120
DURATION_MIN = 0
DURATION_MAX = 5

def hheader(x):
    print("#########################################")
    print("### {}".format(x))
    print("#########################################")

# Magenta dependencies:
# https://github.com/magenta/magenta

# Magenta uses pretty_midi to deal with midi files
import pretty_midi

# Step 1 - Preprocess and quantize MIDI datasets

In [25]:
RUN_STEP_1 = True

### Set up folder for step 1
STEP1_FOLDER = add_output_path("step1")
if not os.path.exists(STEP1_FOLDER):
    os.mkdir(STEP1_FOLDER)
add_step1_path = lambda x: "{}/{}".format(STEP1_FOLDER, x)

print(STEP1_FOLDER)
print()

if RUN_STEP_1:
    filenames = [str(path) for path in Path(INPUT_FOLDER).rglob("*.mid") if MUSIC_FILE_REGEX in str(path)]
    ### Important note: process all tracks, but note that the
    ### primary tracks of interest are the PIANO tracks (and the other
    ### tracks primarily used for beat tracking.)
    print("Available filenames:")
    print("\n".join(filenames))
    print()
    
    ### Filter for core music streams and their tagging information
    core_filenames = [f for f in filenames if STRUCTURAL_BREAK_REGEX not in f]
    core_filenames_legend = defaultdict(lambda: defaultdict(str))
    for f in core_filenames:
        f_seed = f.replace(".mid", "") + "_{}".format(STRUCTURAL_BREAK_REGEX)
        for f2 in filenames:
            if f_seed in f2:
                ts_type = f2.replace(".mid", "").split("_{}_".format(STRUCTURAL_BREAK_REGEX))[-1]
                core_filenames_legend[f][ts_type] = f2
    core_filenames_legend = {k: dict(v) for k, v in dict(core_filenames_legend).items()}

    ### Most cases: make sure only 1 midi stream for MUSIC_FILE_REGEX
    if USE_ONLY_FIRST_MIDI_STREAM:
        if len(core_filenames) > 1:
            raise Exception("Multiple files returned! Check you only have 1 file for MUSIC_FILE_REGEX in final_datasets/.")
        core_filenames = [core_filenames[0]]
    
    all_raw_dfs = []
    all_agg_dfs = []
    for cf, curr_file in enumerate(filenames):
        curr_file_fmt = curr_file.replace("\\", "/").replace(INPUT_FOLDER, "")
        curr_file_out_seed = curr_file_fmt.replace("/","__").replace(".midi", "")
        print("Processing file {} / {}: {}".format(cf+1, len(filenames), curr_file_fmt))
        use_for_analysis = (STRUCTURAL_BREAK_REGEX not in curr_file_fmt)
        if not use_for_analysis:
            print("(skipping for analysis)")
        curr_midi = pretty_midi.PrettyMIDI(curr_file)
        
        ### get solo piano part
        solo_piano_part = curr_midi.instruments[0]
        df_notes = pd.DataFrame([(n.start, n.end, n.pitch, n.velocity, n.duration) for n in solo_piano_part.notes],
                                columns=['start', 'end', 'pitch', 'velocity', 'duration'])
        
        ### specify MUSIC_FILE_REGEX, movement, pianist, date here.
        # [MUSIC_FILE_REGEX, movement, pianist, date] = curr_file.replace("\\", "/").replace(INPUT_FOLDER, "").split("_")
        [music_info, movement, pianist, date] = [
#             "(n/a)", "(n/a)", "(n/a)", "(n/a)"
            MUSIC_FILE_REGEX, PIECE_NAME, PIANIST, DATE
        ]
        
        midi_filename = curr_file_fmt
        df_meta = pd.DataFrame([music_info, movement, pianist, pd.to_datetime(date.replace("-SMD.mid", "")), midi_filename]).T
        df_meta.columns=["music_info", "movement", "pianist", "date", "fp"]
        df_curr = pd.merge(df_notes, df_meta, how="cross")
        df_curr = df_curr.sort_values(by='start').reset_index(drop=True)
        df_curr.to_csv(add_step1_path(curr_file_out_seed + "_raw_" + ".csv"))
        
        if use_for_analysis:
            all_raw_dfs.append(df_curr)
        
        """ Time bar file + metadata """
        # Do not adjust start
        EVERY_N_SEC = 1
        df_curr_time = df_curr.copy()
        df_curr_time['start_sec'] =  (df_curr_time['start'] / EVERY_N_SEC).apply(np.floor).astype(int) * EVERY_N_SEC
        tmp = df_curr_time.select_dtypes(include=[np.number])
        df_curr_time.loc[:, tmp.columns] = np.round(tmp, decimals=5)
        def str_concat(x): return(','.join([str(s) for s in x]))
        df_curr_time_agg = df_curr_time.groupby(['start_sec'], as_index=False).agg({
            'start': [len, np.min, np.mean, np.median, np.max, str_concat],
            'end': [np.min, np.mean, np.median, np.max, str_concat],
            'pitch': [np.min, np.mean, np.median, np.max, str_concat],
            'velocity': [np.min, np.mean, np.median, np.max, str_concat],
            'duration': [np.min, np.mean, np.median, np.max, str_concat],
            # metadata features - same for all observations
            'music_info': [pd.Series.mode],
            'movement': [pd.Series.mode],
            'pianist': [pd.Series.mode],
            'date': [pd.Series.mode],
            'fp': [pd.Series.mode]})
        df_curr_time_agg.columns = ['_'.join([cc for cc in c if len(cc) > 0])
                                    .replace("amin", "min").replace("amax", "max").replace("start_len", "n_notes")
                                    for c in list(df_curr_time_agg.columns)]
        df_curr_time_agg = pd.merge(df_curr_time_agg, # add missing times
                                # Always start from second 1
                                 pd.DataFrame(range(1, df_curr_time_agg.shape[0], 1), columns=["start_sec"]), on="start_sec",
                                how='right').sort_values(by="start_sec")
        df_curr_time_agg.to_csv(add_step1_path(curr_file_out_seed + "_agg_" + ".csv"))
        if use_for_analysis:
            all_agg_dfs.append(df_curr_time_agg)

results/core1/saarland/Mozart_KV265_006/pitch_mean/step1

Available filenames:
final_datasets\Mozart_KV265_006_20110315-SMD.mid

Processing file 1 / 1: /Mozart_KV265_006_20110315-SMD.mid


In [26]:
### Write out dataframes (<3 min) of music
fp_raw_combined = add_step1_path("maestro_raw.csv")
fp_agg_combined = add_step1_path("maestro_agg.csv")
if RUN_STEP_1:
    pd.concat([df.assign(piece_num=(dx+1)) for dx, df in enumerate(all_raw_dfs)]).to_csv(fp_raw_combined, index=False)
    pd.concat([df.assign(piece_num=(dx+1)) for dx, df in enumerate(all_agg_dfs)]).to_csv(fp_agg_combined, index=False)

# Step 2 - read in music stream data

In [27]:
RUN_STEP_2 = True

### Set up folder for step 2
STEP2_FOLDER = add_output_path("step2")
if not os.path.exists(STEP2_FOLDER):
    os.mkdir(STEP2_FOLDER)
add_step2_path = lambda x: "{}/{}".format(STEP2_FOLDER, x)

print(STEP2_FOLDER)

results/core1/saarland/Mozart_KV265_006/pitch_mean/step2


In [28]:
### <2 min
df_maestro_raw = pd.read_csv(fp_raw_combined)
df_maestro_agg = pd.read_csv(fp_agg_combined)

print(df_maestro_raw.shape)
display(df_maestro_raw.head())
print(df_maestro_agg.shape)
display(df_maestro_agg.head())
# all pieces start at least at start second 1 or later
# (df_maestro_agg.start_min - df_maestro_agg.start_sec).describe()

(4187, 11)


,start,end,pitch,velocity,duration,music_info,movement,pianist,date,fp,piece_num
0,0.968,1.200,72,39,0.232,Mozart_KV265_006,Mozart_KV265_006,Saarland,2021-05-01,/Mozart_KV265_006_20110315-SMD.mid,1
1,0.978,1.418,48,31,0.440,Mozart_KV265_006,Mozart_KV265_006,Saarland,2021-05-01,/Mozart_KV265_006_20110315-SMD.mid,1
2,1.483,1.572,72,59,0.089,Mozart_KV265_006,Mozart_KV265_006,Saarland,2021-05-01,/Mozart_KV265_006_20110315-SMD.mid,1
3,1.505,1.993,60,36,0.487,Mozart_KV265_006,Mozart_KV265_006,Saarland,2021-05-01,/Mozart_KV265_006_20110315-SMD.mid,1
4,1.995,2.230,79,61,0.235,Mozart_KV265_006,Mozart_KV265_006,Saarland,2021-05-01,/Mozart_KV265_006_20110315-SMD.mid,1


(466, 33)


,start_sec,n_notes,start_min,start_mean,start_median,start_max,start_str_concat,end_min,end_mean,end_median,end_max,end_str_concat,pitch_min,pitch_mean,pitch_median,pitch_max,pitch_str_concat,velocity_min,velocity_mean,velocity_median,velocity_max,velocity_str_concat,duration_min,duration_mean,duration_median,duration_max,duration_str_concat,music_info_mode,movement_mode,pianist_mode,date_mode,fp_mode,piece_num
0,1,4.000,1.483,1.745,1.750,1.998,"1.48333,1.50521,1.99479,1.99792",1.572,2.071,2.111,2.490,"1.57188,1.99271,2.23021,2.48958",60.000,68.750,68.000,79.000,"72,60,79,64",36.000,48.250,48.000,61.000,"59,36,61,37",0.089,0.326,0.361,0.492,"0.08854,0.4875,0.23542,0.49167",Mozart_KV265_006,Mozart_KV265_006,Saarland,2021-05-01,/Mozart_KV265_006_20110315-SMD.mid,1
1,2,4.000,2.460,2.718,2.716,2.980,"2.46042,2.49375,2.9375,2.98021",2.661,3.096,3.129,3.467,"2.66146,3.04375,3.21354,3.46667",60.000,71.250,72.000,81.000,"79,60,81,65",38.000,51.500,52.000,64.000,"64,41,63,38",0.201,0.378,0.381,0.550,"0.20104,0.55,0.27604,0.48646",Mozart_KV265_006,Mozart_KV265_006,Saarland,2021-05-01,/Mozart_KV265_006_20110315-SMD.mid,1
2,3,4.000,3.419,3.683,3.690,3.934,"3.41875,3.45729,3.92292,3.93438",3.597,4.040,4.088,4.389,"3.59688,3.97708,4.19792,4.38854",60.000,71.000,71.500,81.000,"81,60,79,64",36.000,48.000,47.000,62.000,"62,36,58,36",0.178,0.357,0.365,0.520,"0.17813,0.51979,0.275,0.45417",Mozart_KV265_006,Mozart_KV265_006,Saarland,2021-05-01,/Mozart_KV265_006_20110315-SMD.mid,1
3,4,4.000,4.407,4.644,4.646,4.878,"4.40729,4.41771,4.87396,4.87812",4.592,4.977,4.984,5.348,"4.59167,4.82396,5.34792,5.14479",60.000,69.500,69.500,79.000,"79,60,62,77",33.000,49.500,52.500,60.000,"60,33,46,59",0.184,0.333,0.336,0.474,"0.18438,0.40625,0.47396,0.26667",Mozart_KV265_006,Mozart_KV265_006,Saarland,2021-05-01,/Mozart_KV265_006_20110315-SMD.mid,1
4,5,4.000,5.360,5.620,5.621,5.877,"5.36042,5.37812,5.86458,5.87708",5.537,5.966,5.981,6.367,"5.5375,5.85729,6.10417,6.36667",59.000,68.000,68.000,77.000,"77,59,76,60",33.000,46.250,46.000,60.000,"60,37,55,33",0.177,0.346,0.359,0.490,"0.17708,0.47917,0.23958,0.48958",Mozart_KV265_006,Mozart_KV265_006,Saarland,2021-05-01,/Mozart_KV265_006_20110315-SMD.mid,1


Raw is basically a dataframe of all the raw irregular time series (concatenated), while agg is basically a processed version that is by every 1 second, 2 second, etc. (still aligned with raw file timestamps, NOT realigned).

In [29]:
""" Agg: show number of notes per piece and time bars per piece """

print("Number of notes per piece: ")
display(df_maestro_raw.groupby(['piece_num']).size().head())

print("Time bars (seconds) per piece: ")
display(df_maestro_agg.groupby(['piece_num']).size().head())

Number of notes per piece: 


piece_num
1    4187
dtype: int64

Time bars (seconds) per piece: 


piece_num
1    466
dtype: int64

In [30]:
""" Visualization of the raw time series """
basic_viz = px.line(df_maestro_agg, x="start_sec", y=TIME_SERIES_VARIABLE, template="plotly_dark")
basic_viz.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(basic_viz)
basic_viz.write_image(add_step2_path("agg_time_series.png"), width=FIG_WIDTH, height=FIG_HEIGHT)

# Step 3 - modeling, prediction and change point detection

1. Kalman-ARIMA filter + CUSUM
2. Kalman-ARIMA filter + BOCD (Bayesian Online Changepoint Detection)
3. Bayesian linear regression + CUSUM
4. Bayesian linear regression + BOCD
5. Ridge regression + CUSUM
6. Ridge regression + BOCD
7. Gaussian process regression + CUSUM
8. Gaussian process regression + BOCD

Note: all models learn sequentially except for the Gaussian process methods (7, 8) which are trained in minibatch.

In [31]:
""" Modeling prep """

### Set up folder for step 3
STEP3_FOLDER = add_output_path("step3")
if not os.path.exists(STEP3_FOLDER):
    os.mkdir(STEP3_FOLDER)
add_step3_path = lambda x: "{}/{}".format(STEP3_FOLDER, x)

print(STEP3_FOLDER)

results/core1/saarland/Mozart_KV265_006/pitch_mean/step3


In [32]:
### Store all models
df_all_modeling_results = list()

### Set the time series to model and predict.
if (TIME_SERIES_VARIABLE not in df_maestro_agg.columns):
    raise Exception("!!! Time series not found.")
ts = df_maestro_agg['{}'.format(TIME_SERIES_VARIABLE)].ravel()
N = ts.size
print(ts.shape)
ts[:10]

### Set up time series x_obs to use
x_obs = np.c_[ts]

""" Common parameters across models """

REG_L = 20
BURN_IN_PERIOD = max(int(np.floor(len(ts) / 10)), REG_L)
# Number of steps to run for (when prototyping)
PROTOTYPE_TIME_STEPS = int(min(5000, len(ts) - REG_L))

(466,)


### Model 1: Kalman filter + CUSUM

Source:
- Puhm, Martin, et al. "A Near Real-Time Method for Forest Change Detection Based on a Structural Time Series Model and the Kalman Filter." Remote Sensing 12.19 (2020): 3135.

In [33]:
""" Model setup """

import numpy as np
import bocd
import scipy as sc
from scipy import stats
from scipy.stats import multivariate_normal
from statsmodels.tsa.stattools import acovf
from scipy.ndimage.interpolation import shift

""" Kalman filter parameters """

LxL = (REG_L, REG_L)
Lx1 = (REG_L, 1)
Lx1_T = (1, REG_L)

# unobserved
zt = np.zeros(Lx1)
At = np.eye(REG_L)
wt = np.zeros(Lx1)
Rt = np.zeros(LxL)

# observed
xt = np.zeros((1,1))
Ct = np.zeros((1,REG_L)) # will hold the ARIMA lags
vt = np.zeros((1,1))
Qt = np.zeros((1,1))

### Store states and predictions
Zthat = [] # will be NxL
ZthatSize = (N, REG_L)
Xthat = [] # will be Nx1
XthatSize = (N, 1)
df_pred_vs_actual = []

### initialize until REG_L
zt = np.ones(Lx1) * (1/REG_L)
for j in range(REG_L):
    Zthat.append(zt)
Rt = np.diag(np.random.gamma(1,0.1,size=REG_L)) # covar. mat is symmetric
Qt = np.random.gamma(3, 4, size=(1,1))
Pt = Rt

""" CUSUM parameters """
threshold = 25 # increase for more strict
drift = 0
gp, gn = np.zeros(N), np.zeros(N)
ta, tai, taf = np.array([[], [], []], dtype=int)
tap, tan = 0, 0
amp = np.array([])
residuals = np.empty((N, 1))
residuals[:] = np.nan
num_residuals_initialized = 0

""" Train sequentially """
timerange = range(REG_L, N-1) # data starts at 0 so start analysis at L for L lags
timerange = range(REG_L, REG_L + PROTOTYPE_TIME_STEPS)
for i in timerange:
    if (i % 10000 == 0): print("Training observation {} / {} ...".format(i+1, n-1))
    # will predict x ~ z basically
    xt = ts[i].reshape(xt.shape)
    Ct = np.array(ts[(i-REG_L):i]).reshape(Ct.shape)
    
    # For unobserved (covariates), impute mean if lags are NaN
    # If not possible then put NA and continue
    if np.isnan([Ct]).any():
        # impute mean if possible, if not then do NA
        Ct_nonna = Ct[~np.isnan(Ct)]
        if len(Ct_nonna) < 1:
            df_pred_vs_actual.append([i, y, np.nan])
            continue
        else:
            Ct_impute = np.copy(Ct)
            Ct_impute[np.isnan(Ct_impute)] = np.nanmean(Ct_impute)
            Ct = Ct_impute

    # For missing outcome xt, update array and skip
    if (np.isnan([xt]).any()):
        df_pred_vs_actual.append([i, xt, np.nan]) # NAN prediction
        continue

    """ Run Kalman filter """
    
    ### Prediction
    zt_hat = At.dot(zt)
    Pt = At.dot(Pt).dot(At.T) + Rt
    xt_hat = Ct.dot(zt_hat)
    
    # compare against ground truth
    residual = xt - xt_hat
    
    ### Update
    Kt = Pt.dot(Ct.T).dot(sc.linalg.inv(Ct.dot(Pt).dot(Ct.T) + Qt))
    zt_hat = zt_hat + Kt.dot(xt - Ct.dot(zt_hat))
    Pt = Pt - Kt.dot(Ct).dot(Pt)
    
    ### Store predictions
    Zthat.append(zt_hat)
    Xthat.append(xt_hat)
    df_pred_vs_actual.append([i, float(xt.item()), float(xt_hat.item())])
    residuals[i] = (xt - xt_hat).item()
    num_residuals_initialized += 1
    
    """ CUSUM (classical) """
    
    # (only run if already have 1 or more residuals calculated, and
    # don't have all NAN residuals for the last lags)
    if num_residuals_initialized < 2 or np.isnan(residuals[(i-REG_L):i]).all():
        continue
    
    # Auto-estimate drift to be (1/2)*expected change
    # aka mean of the last REG_LAG residuals.
    drift = np.nanmean(residuals[(i-REG_L):i]) / 2
    drift = 0
    
    # Auto-estimate threshold
    
    s = np.array(residuals[i]) - np.array(residuals[i-1])
    # do not train on NaNs
    if (np.isnan(s).any()):
        continue
    gp[i] = gp[i-1] + s - drift  # cumulative sum for + change
    gn[i] = gn[i-1] - s - drift  # cumulative sum for - change
    if gp[i] < 0:
        gp[i], tap = 0, i
    if gn[i] < 0:
        gn[i], tan = 0, i
    if gp[i] > threshold or gn[i] > threshold:  # change detected!
        ta = np.append(ta, i)    # alarm index
        tai = np.append(tai, tap if gp[i] > threshold else tan)  # start
        gp[i], gn[i] = 0, 0      # reset alarm

""" Plot """
df_plt = pd.DataFrame(df_pred_vs_actual)
df_plt.columns = ['t', 'actual', 'pred']
df_plt['actual'] = df_plt['actual'].astype(float)
df_plt['resid'] = df_plt['actual'] - df_plt['pred']
df_resid = pd.DataFrame({"t": ta, "break": 1})
df_plt2 = pd.merge(df_plt, df_resid, on="t", how="left")
df_plt2[['break']] = df_plt2[['break']].fillna(0)

fig = px.line(df_plt2, x="t", y=['actual', 'pred'], template="plotly_dark")
fig.add_scatter(x=df_plt2['t'], y=df_plt2['actual'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig.update_layout(title="Actual vs. predicted")
fig.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig)

fig2 = px.line(df_plt2, x="t", y=['resid'], template="plotly_dark")
fig2.add_scatter(x=df_plt2['t'], y=df_plt2['resid'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig2.update_layout(title="Residuals")
fig2.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig2)

# fig.write_image(add_("model1_kalman_filter_cusum__plt1_actual_vs_pred.png"))
# fig2.write_image(add_output_path("model1_kalman_filter_cusum__plt2_residuals.png"))

### Store modeling results
df_plt2['mse'] = np.mean((df_plt2['actual'] - df_plt2['pred'])**2)
df_plt2['rmse'] = np.sqrt(np.mean((df_plt2['actual'] - df_plt2['pred'])**2))
df_plt2['mae'] = np.mean(np.abs(df_plt2['pred'] - df_plt2['actual']))
df_plt2['mape'] = np.mean(100 * np.abs( 1 - df_plt2['pred'] / df_plt2['actual'] ))
df_plt2_final = df_plt2.assign(model_no=1, reg_model="Kalman Filter", change_model="CUSUM")
df_all_modeling_results.append(df_plt2_final)

### Save plots
plt_seed = '_'.join(map(str,
    df_plt2_final[['model_no', 'reg_model', 'change_model']].iloc[0,].values))
fig.write_image(add_step3_path("actual_vs_pred__{}.png".format(plt_seed)),
               width=FIG_WIDTH, height=FIG_HEIGHT)
fig2.write_image(add_step3_path("residuals__{}.png".format(plt_seed)),
                width=FIG_WIDTH, height=FIG_HEIGHT)

### Model 2: Kalman filter + BOCD


In [34]:
""" Model setup """

import numpy as np
import bocd
import scipy as sc
from scipy import stats
from scipy.stats import multivariate_normal
from statsmodels.tsa.stattools import acovf
from scipy.ndimage.interpolation import shift

""" Kalman filter parameters """

LxL = (REG_L, REG_L)
Lx1 = (REG_L, 1)
Lx1_T = (1, REG_L)

# unobserved
zt = np.zeros(Lx1)
At = np.eye(REG_L)
wt = np.zeros(Lx1)
Rt = np.zeros(LxL)

# observed
xt = np.zeros((1,1))
Ct = np.zeros((1,REG_L)) # will hold the ARIMA lags
vt = np.zeros((1,1))
Qt = np.zeros((1,1))

### Store states and predictions
Zthat = [] # will be NxL
ZthatSize = (N, REG_L)
Xthat = [] # will be Nx1
XthatSize = (N, 1)
df_pred_vs_actual = []

### initialize until REG_L
zt = np.ones(Lx1) * (1/REG_L)
for j in range(REG_L):
    Zthat.append(zt)
Rt = np.diag(np.random.gamma(1,0.1,size=REG_L)) # covar. mat is symmetric
Qt = np.random.gamma(3, 4, size=(1,1))
Pt = Rt

""" BOCD parameters """
# https://github.com/hildensia/bayesian_changepoint_detection/blob/
# master/bayesian_changepoint_detection/online_changepoint_detection.py

bc = bocd.BayesianOnlineChangePointDetection(bocd.ConstantHazard(300),
    bocd.StudentT(mu=0, kappa=1, alpha=1, beta=1))

rt_mle = np.empty(N)
residuals = np.empty((N, 1))
residuals[:] = np.nan
num_residuals_initialized = 0

""" Train sequentially """
timerange = range(REG_L, N-1) # data starts at 0 so start analysis at L for L lags
timerange = range(REG_L, REG_L + PROTOTYPE_TIME_STEPS)
for i in timerange:
    if (i % 10000 == 0): print("Training observation {} / {} ...".format(i+1, n-1))
    # will predict x ~ z basically
    xt = ts[i].reshape(xt.shape)
    Ct = np.array(ts[(i-REG_L):i]).reshape(Ct.shape)
    
    # For unobserved (covariates), impute mean if lags are NaN
    # If not possible then put NA and continue
    if np.isnan([Ct]).any():
        # impute mean if possible, if not then do NA
        Ct_nonna = Ct[~np.isnan(Ct)]
        if len(Ct_nonna) < 1:
            df_pred_vs_actual.append([i, y, np.nan])
            continue
        else:
            Ct_impute = np.copy(Ct)
            Ct_impute[np.isnan(Ct_impute)] = np.nanmean(Ct_impute)
            Ct = Ct_impute

    # For missing outcome xt, update array and skip
    if (np.isnan([xt]).any()):
        df_pred_vs_actual.append([i, xt, np.nan]) # NAN prediction
        continue

    """ Run Kalman filter """
    
    ### Prediction
    zt_hat = At.dot(zt)
    Pt = At.dot(Pt).dot(At.T) + Rt
    xt_hat = Ct.dot(zt_hat)
    
    # compare against ground truth
    residual = xt - xt_hat
    
    ### Update
    Kt = Pt.dot(Ct.T).dot(sc.linalg.inv(Ct.dot(Pt).dot(Ct.T) + Qt))
    zt_hat = zt_hat + Kt.dot(xt - Ct.dot(zt_hat))
    Pt = Pt - Kt.dot(Ct).dot(Pt)
    
    ### Store predictions
    Zthat.append(zt_hat)
    Xthat.append(xt_hat)
    df_pred_vs_actual.append([i, float(xt.item()), float(xt_hat.item())])
    residuals[i] = (xt - xt_hat).item()
    num_residuals_initialized += 1
    
    """ BOCD (Bayesian Online Changepoint Detection) """
    
    # (only run if already have 1 or more residuals calculated, and
    # don't have all NAN residuals for the last lags)
    if num_residuals_initialized < 2 or np.isnan(residuals[(i-REG_L):i]).all():
        continue
    if np.isnan(residuals[i]).all():
        continue
    
    # update
    bc.update(residuals[i])
    rt_mle[i] = bc.rt

""" Plot """
df_plt = pd.DataFrame(df_pred_vs_actual)
df_plt.columns = ['t', 'actual', 'pred']
df_plt['actual'] = df_plt['actual'].astype(float)
df_plt['resid'] = df_plt['actual'] - df_plt['pred']
# add change points
index_changes = np.where(np.diff(rt_mle.flatten())<0)[0]
df_plt2 = pd.merge(df_plt, pd.DataFrame({"t": index_changes, "break": 1}), on="t", how="left")
df_plt2[['break']] = df_plt2[['break']].fillna(0)

fig = px.line(df_plt2, x="t", y=['actual', 'pred'], template="plotly_dark")
fig.add_scatter(x=df_plt2['t'], y=df_plt2['actual'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig.update_layout(title="Actual vs. predicted")
fig.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig)

fig2 = px.line(df_plt2, x="t", y=['resid'], template="plotly_dark")
fig2.add_scatter(x=df_plt2['t'], y=df_plt2['resid'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig2.update_layout(title="Residuals")
fig2.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig2)

### Store modeling results
df_plt2['mse'] = np.mean((df_plt2['actual'] - df_plt2['pred'])**2)
df_plt2['rmse'] = np.sqrt(np.mean((df_plt2['actual'] - df_plt2['pred'])**2))
df_plt2['mae'] = np.mean(np.abs(df_plt2['pred'] - df_plt2['actual']))
df_plt2['mape'] = np.mean(100 * np.abs( 1 - df_plt2['pred'] / df_plt2['actual'] ))
df_plt2_final = df_plt2.assign(model_no=2, reg_model="Kalman Filter", change_model="BOCD")
df_all_modeling_results.append(df_plt2_final)

### Save plots
plt_seed = '_'.join(map(str,
    df_plt2_final[['model_no', 'reg_model', 'change_model']].iloc[0,].values))
fig.write_image(add_step3_path("actual_vs_pred__{}.png".format(plt_seed)),
               width=FIG_WIDTH, height=FIG_HEIGHT)
fig2.write_image(add_step3_path("residuals__{}.png".format(plt_seed)),
                width=FIG_WIDTH, height=FIG_HEIGHT)

### Model 3: Bayesian linear regression + CUSUM

Directly adjusted.

- http://www.prodsyse.com/Bayes-Adj%20Cusum2.pdf
- https://cran.r-project.org/web/packages/spcadjust/vignettes/CUSUM_LinearRegression.html
- https://github.com/BMClab/BMC/blob/master/functions/detect_cusum.py

In [35]:
""" Model setup """

import numpy as np
from scipy import stats

# adapted from https://maxhalford.github.io/blog/bayesian-linear-regression/
# + computer class code
# n_features = REG_L
# alpha = 0.1 
# beta = 1
# mean = np.zeros(n_features)
# cov_inv = np.identity(n_features) / alpha

class BayesLinReg:
    def __init__(self, n_features, alpha, beta):
        self.n_features = n_features
        self.alpha = alpha
        self.beta = beta
        self.mean = np.zeros(n_features)
        self.cov_inv = np.identity(n_features) / alpha
    def learn(self, x, y):
        # Update the inverse covariance matrix (Bishop eq. 3.51)
        cov_inv = self.cov_inv + self.beta * np.outer(x, x)
        # Update the mean vector (Bishop eq. 3.50)
        cov = np.linalg.inv(cov_inv)
        mean = cov @ (self.cov_inv @ self.mean + self.beta * y * x)
        self.cov_inv = cov_inv
        self.mean = mean
        return self
    def predict(self, x):
        # Obtain the predictive mean (Bishop eq. 3.58)
        y_pred_mean = x @ self.mean
        # Obtain the predictive variance (Bishop eq. 3.59)
        w_cov = np.linalg.inv(self.cov_inv)
        y_pred_var = 1 / self.beta + x @ w_cov @ x.T
        return stats.norm(loc=y_pred_mean, scale=y_pred_var ** .5)
    @property
    def weights_dist(self):
        cov = np.linalg.inv(self.cov_inv)
        return stats.multivariate_normal(mean=self.mean, cov=cov)

# Parameters here
blr = BayesLinReg(n_features=REG_L, alpha=0.1, beta=1)
df_pred_vs_actual = []
residuals = np.zeros(x_obs.shape)

### CUSUM parameters
threshold = 15 # increase for more strict
drift = 0
gp, gn = np.zeros(x_obs.size), np.zeros(x_obs.size)
ta, tai, taf = np.array([[], [], []], dtype=int)
tap, tan = 0, 0
amp = np.array([])

""" Train sequentially """
timerange = range(REG_L, x_obs.size-1) # data starts at 0 so start analysis at L for L lags
timerange = range(REG_L, REG_L + PROTOTYPE_TIME_STEPS)
for i in timerange:
    if (i % 10000 == 0): print("Training observation {} / {} ...".format(i+1, n-1))
#     y = np.array(x_obs[i])
    y = x_obs[i].tolist()[0]
    x = np.array(x_obs[(i-REG_L):i]).reshape(-1,)
    
    # impute mean if lags are NaN, if not possible then put NA and continue
    if np.isnan([x]).any():
        # impute mean if possible, if not then do NA
        x_nonna = x[~np.isnan(x)]
        if len(x_nonna) < 1:
            df_pred_vs_actual.append([i, y, np.nan])
            continue
        else:
            x_impute = np.copy(x)
            x_impute[np.isnan(x_impute)] = np.nanmean(x_impute)
            x = x_impute
    
    # skip training if output is NA
    if (np.isnan([y]).any()):
        continue

    if (i <= BURN_IN_PERIOD):
        # just train
        blr.learn(x, y)
        continue

    yhat = blr.predict(x).mean()
    if (np.isnan(yhat).any()):
        break
    df_pred_vs_actual.append([i, y, yhat])
    blr.learn(x, y)
    residuals[i] = y - yhat
    
    """ CUSUM """

    # CUSUM on the (online) residuals
    # (only run if already have 1 or more residuals calculated)
    r = residuals
    if (r[i-1] == 0):
        continue
    
    # Auto-estimate drift to be (1/2)*expected change
    # aka mean of the last REG_LAG residuals.
    drift = np.mean(residuals[(i-REG_L):i]) / 2
    
    # Auto-estimate threshold
    
    s = np.array(r[i]) - np.array(r[i-1])
    gp[i] = gp[i-1] + s - drift  # cumulative sum for + change
    gn[i] = gn[i-1] - s - drift  # cumulative sum for - change
    if gp[i] < 0:
        gp[i], tap = 0, i
    if gn[i] < 0:
        gn[i], tan = 0, i
    if gp[i] > threshold or gn[i] > threshold:  # change detected!
        ta = np.append(ta, i)    # alarm index
        tai = np.append(tai, tap if gp[i] > threshold else tan)  # start
        gp[i], gn[i] = 0, 0      # reset alarm


""" Plot """
df_plt = pd.DataFrame(df_pred_vs_actual)
df_plt.columns = ['t', 'actual', 'pred']
df_plt['resid'] = df_plt['actual'] - df_plt['pred']
df_plt2 = pd.merge(df_plt, pd.DataFrame({"t": ta, "break": 1}), on="t", how="left").fillna(0)
fig = px.line(df_plt2, x="t", y=['actual', 'pred'], template="plotly_dark")
fig.add_scatter(x=df_plt2['t'], y=df_plt2['actual'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig.update_layout(title="Actual vs. predicted")
fig.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig)

fig2 = px.line(df_plt2, x="t", y=['resid'], template="plotly_dark")
fig2.add_scatter(x=df_plt2['t'], y=df_plt2['resid'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig2.update_layout(title="Residuals")
fig2.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig2)

### Store modeling results
df_plt2['mse'] = np.mean((df_plt2['actual'] - df_plt2['pred'])**2)
df_plt2['rmse'] = np.sqrt(np.mean((df_plt2['actual'] - df_plt2['pred'])**2))
df_plt2['mae'] = np.mean(np.abs(df_plt2['pred'] - df_plt2['actual']))
df_plt2['mape'] = np.mean(100 * np.abs( 1 - df_plt2['pred'] / df_plt2['actual'] ))
df_plt2_final = df_plt2.assign(model_no=3, reg_model="Bayesian LR", change_model="CUSUM")
df_all_modeling_results.append(df_plt2_final)

### Save plots
plt_seed = '_'.join(map(str,
    df_plt2_final[['model_no', 'reg_model', 'change_model']].iloc[0,].values))
fig.write_image(add_step3_path("actual_vs_pred__{}.png".format(plt_seed)),
               width=FIG_WIDTH, height=FIG_HEIGHT)
fig2.write_image(add_step3_path("residuals__{}.png".format(plt_seed)),
                width=FIG_WIDTH, height=FIG_HEIGHT)

### Model 4: Bayesian linear regression + BOCD

https://arxiv.org/pdf/0710.3742.pdf

In [36]:
""" Model setup """

import numpy as np
from scipy import stats

# adapted from https://maxhalford.github.io/blog/bayesian-linear-regression/
# + computer class code
# n_features = REG_L
# alpha = 0.1 
# beta = 1
# mean = np.zeros(n_features)
# cov_inv = np.identity(n_features) / alpha

class BayesLinReg:
    def __init__(self, n_features, alpha, beta):
        self.n_features = n_features
        self.alpha = alpha
        self.beta = beta
        self.mean = np.zeros(n_features)
        self.cov_inv = np.identity(n_features) / alpha
    def learn(self, x, y):
        # Update the inverse covariance matrix (Bishop eq. 3.51)
        cov_inv = self.cov_inv + self.beta * np.outer(x, x)
        # Update the mean vector (Bishop eq. 3.50)
        cov = np.linalg.inv(cov_inv)
        mean = cov @ (self.cov_inv @ self.mean + self.beta * y * x)
        self.cov_inv = cov_inv
        self.mean = mean
        return self
    def predict(self, x):
        # Obtain the predictive mean (Bishop eq. 3.58)
        y_pred_mean = x @ self.mean
        # Obtain the predictive variance (Bishop eq. 3.59)
        w_cov = np.linalg.inv(self.cov_inv)
        y_pred_var = 1 / self.beta + x @ w_cov @ x.T
        return stats.norm(loc=y_pred_mean, scale=y_pred_var ** .5)
    @property
    def weights_dist(self):
        cov = np.linalg.inv(self.cov_inv)
        return stats.multivariate_normal(mean=self.mean, cov=cov)

# Parameters here
blr = BayesLinReg(n_features=REG_L, alpha=0.1, beta=1)
df_pred_vs_actual = []
residuals = np.zeros(x_obs.shape)

### BOCD 
bc = bocd.BayesianOnlineChangePointDetection(bocd.ConstantHazard(300), bocd.StudentT(mu=0, kappa=1, alpha=1, beta=1))
rt_mle = np.empty(x_obs.shape)

""" Train sequentially """
timerange = range(REG_L, x_obs.size-1) # data starts at 0 so start analysis at L for L lags
timerange = range(REG_L, REG_L + PROTOTYPE_TIME_STEPS)
for i in timerange:
    if (i % 10000 == 0): print("Training observation {} / {} ...".format(i+1, n-1))
#     y = np.array(x_obs[i])
    y = x_obs[i].tolist()[0]
    x = np.array(x_obs[(i-REG_L):i]).reshape(-1,)
    
    # impute mean if lags are NaN, if not possible then put NA and continue
    if np.isnan([x]).any():
        # impute mean if possible, if not then do NA
        x_nonna = x[~np.isnan(x)]
        if len(x_nonna) < 1:
            df_pred_vs_actual.append([i, y, np.nan])
            continue
        else:
            x_impute = np.copy(x)
            x_impute[np.isnan(x_impute)] = np.nanmean(x_impute)
            x = x_impute
    
    # skip training if output is NA
    if (np.isnan([y]).any()):
        continue

    if (i <= BURN_IN_PERIOD):
        # just train
        blr.learn(x, y)
        continue

    yhat = blr.predict(x).mean()
    if (np.isnan(yhat).any()):
        break
    df_pred_vs_actual.append([i, y, yhat])
    blr.learn(x, y)
    residuals[i] = y - yhat
    
    """ Change point detection """
    
    r = residuals
    
    bc.update(r[i])
    rt_mle[i] = bc.rt


""" Plot """
df_plt = pd.DataFrame(df_pred_vs_actual)
df_plt.columns = ['t', 'actual', 'pred']
df_plt['resid'] = df_plt['actual'] - df_plt['pred']
# add change points
index_changes = np.where(np.diff(rt_mle.flatten())<0)[0]
df_plt2 = pd.merge(df_plt, pd.DataFrame({"t": index_changes, "break": 1}), on="t", how="left").fillna(0)

fig = px.line(df_plt2, x="t", y=['actual', 'pred'], template="plotly_dark")
fig.add_scatter(x=df_plt2['t'], y=df_plt2['actual'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig.update_layout(title="Actual vs. predicted")
fig.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig)

fig2 = px.line(df_plt2, x="t", y=['resid'], template="plotly_dark")
fig2.add_scatter(x=df_plt2['t'], y=df_plt2['resid'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig2.update_layout(title="Residuals")
fig2.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig2)

### Store modeling results
df_plt2['mse'] = np.mean((df_plt2['actual'] - df_plt2['pred'])**2)
df_plt2['rmse'] = np.sqrt(np.mean((df_plt2['actual'] - df_plt2['pred'])**2))
df_plt2['mae'] = np.mean(np.abs(df_plt2['pred'] - df_plt2['actual']))
df_plt2['mape'] = np.mean(100 * np.abs( 1 - df_plt2['pred'] / df_plt2['actual'] ))
df_plt2_final = df_plt2.assign(model_no=4, reg_model="Bayesian LR", change_model="BOCD")
df_all_modeling_results.append(df_plt2_final)

### Save plots
plt_seed = '_'.join(map(str,
    df_plt2_final[['model_no', 'reg_model', 'change_model']].iloc[0,].values))
fig.write_image(add_step3_path("actual_vs_pred__{}.png".format(plt_seed)),
               width=FIG_WIDTH, height=FIG_HEIGHT)
fig2.write_image(add_step3_path("residuals__{}.png".format(plt_seed)),
                width=FIG_WIDTH, height=FIG_HEIGHT)

### Model 5: Recursive ridge regression + CUSUM

In [37]:
""" Model setup """

import numpy as np
from scipy import stats

# Parameters here
ridge_beta = np.zeros((REG_L, 1))
model_initialized = False
Qn_initialized = False
df_pred_vs_actual = []
residuals = np.zeros(x_obs.shape)

### CUSUM parameters
threshold = 15 # increase for more strict
drift = 0
gp, gn = np.zeros(x_obs.size), np.zeros(x_obs.size)
ta, tai, taf = np.array([[], [], []], dtype=int)
tap, tan = 0, 0
amp = np.array([])

""" Train sequentially """
timerange = range(REG_L, x_obs.size-1) # data starts at 0 so start analysis at L for L lags
timerange = range(REG_L, REG_L + PROTOTYPE_TIME_STEPS)
x_postprocess = []
y_postprocess = []
for i in timerange:
    if (i % 10000 == 0): print("Training observation {} / {} ...".format(i+1, n-1))
    y = x_obs[i].tolist()[0]
    x = np.array(x_obs[(i-REG_L):i]).reshape(-1,)
    
    # impute mean if lags are NaN, if not possible then put NA and continue
    if np.isnan([x]).any():
        # impute mean if possible, if not then do NA
        x_nonna = x[~np.isnan(x)]
        if len(x_nonna) < 1:
            df_pred_vs_actual.append([i, y, np.nan])
            continue
        else:
            x_impute = np.copy(x)
            x_impute[np.isnan(x_impute)] = np.nanmean(x_impute)
            x = x_impute
    
    # skip training if output is NA
    if (np.isnan([y]).any()):
        continue
    
    if (i <= BURN_IN_PERIOD):
        x_postprocess.append(x)
        y_postprocess.append(y)
        continue
        
    ### initialize ridge
    if not model_initialized:
        X_init = np.vstack(x_postprocess)
        Y_init = np.vstack(y_postprocess)
        
        ### Fit ridge once have enough data
        from sklearn.linear_model import Ridge
        ridge_clf = Ridge(alpha=1)
        ridge_clf.fit(X_init, Y_init)
        ridge_beta = ridge_clf.coef_.reshape(ridge_beta.shape)
        model_initialized = True

    X_curr = np.vstack(x_postprocess)
    Y_curr = np.vstack(y_postprocess)
    
    ### Predict current data point
    yhat = x.dot(ridge_beta)
    if (np.isnan(yhat).any()):
        break
    df_pred_vs_actual.append([i, y, yhat.item()])
    
    ### Train model
    
    # (1/2) - calculate matrix inverse every time (inefficient computation, faster convergence)
#     Qn_inv = sc.linalg.inv(X_curr.T.dot(X_curr) + 1 * np.eye(REG_L))
    # (2/2) - update matrix inverse recursively (efficient computation, slower convergence)
    if not Qn_initialized:
        Qn_inv = sc.linalg.inv(X_curr.T.dot(X_curr) + 1 * np.eye(REG_L))
        Qn_initialized = True
    else:
        Qn_update_num = (Qn_inv.dot(x.reshape(-1, 1).dot(x.reshape(-1, 1).T)).dot(Qn_inv))
        Qn_update_den = (1 + x.reshape(-1,1).T.dot(Qn_inv).dot(x.reshape(-1,1))).item()
        Qn_inv = Qn_inv - (Qn_update_num / Qn_update_den)
    
    # exact - working! (needs longer burn-in period)
    coefQn = (Qn_inv/(1 + x.reshape(-1, 1).T.dot(Qn_inv).dot(x.reshape(-1, 1))))
    
    # approx - does not work well
#     coefQn = (1/(np.power(i, 0.5)) * np.ones(ridge_beta.shape)).T

    xresid = x.T.reshape(-1, 1).dot(y - x.dot(ridge_beta)).reshape(-1, 1)
    # should approximate
    ridge_beta = ridge_beta + coefQn.dot(xresid)
    
    # store residual of actual vs. predicted
    residuals[i] = y - yhat
    
    """ CUSUM """

    # CUSUM on the (online) residuals
    # (only run if already have 1 or more residuals calculated)
    r = residuals
    if (r[i-1] == 0):
        continue
    
    # Auto-estimate drift to be (1/2)*expected change
    # aka mean of the last REG_LAG residuals.
    drift = np.mean(residuals[(i-REG_L):i]) / 2
    
    # Auto-estimate threshold
    
    s = np.array(r[i]) - np.array(r[i-1])
    gp[i] = gp[i-1] + s - drift  # cumulative sum for + change
    gn[i] = gn[i-1] - s - drift  # cumulative sum for - change
    if gp[i] < 0:
        gp[i], tap = 0, i
    if gn[i] < 0:
        gn[i], tan = 0, i
    if gp[i] > threshold or gn[i] > threshold:  # change detected!
        ta = np.append(ta, i)    # alarm index
        tai = np.append(tai, tap if gp[i] > threshold else tan)  # start
        gp[i], gn[i] = 0, 0      # reset alarm


""" Plot """
df_plt = pd.DataFrame(df_pred_vs_actual)
df_plt.columns = ['t', 'actual', 'pred']
df_plt['resid'] = df_plt['actual'] - df_plt['pred']
df_plt2 = pd.merge(df_plt, pd.DataFrame({"t": ta, "break": 1}), on="t", how="left").fillna(0)
fig = px.line(df_plt2, x="t", y=['actual', 'pred'], template="plotly_dark")
fig.add_scatter(x=df_plt2['t'], y=df_plt2['actual'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig.update_layout(title="Actual vs. predicted")
fig.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig)

fig2 = px.line(df_plt2, x="t", y=['resid'], template="plotly_dark")
fig2.add_scatter(x=df_plt2['t'], y=df_plt2['resid'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig2.update_layout(title="Residuals")
fig2.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig2)

### Store modeling results
df_plt2['mse'] = np.mean((df_plt2['actual'] - df_plt2['pred'])**2)
df_plt2['rmse'] = np.sqrt(np.mean((df_plt2['actual'] - df_plt2['pred'])**2))
df_plt2['mae'] = np.mean(np.abs(df_plt2['pred'] - df_plt2['actual']))
df_plt2['mape'] = np.mean(100 * np.abs( 1 - df_plt2['pred'] / df_plt2['actual'] ))
df_plt2_final = df_plt2.assign(model_no=5, reg_model="Recursive ridge regression", change_model="CUSUM")
df_all_modeling_results.append(df_plt2_final)

### Save plots
plt_seed = '_'.join(map(str,
    df_plt2_final[['model_no', 'reg_model', 'change_model']].iloc[0,].values))
fig.write_image(add_step3_path("actual_vs_pred__{}.png".format(plt_seed)),
               width=FIG_WIDTH, height=FIG_HEIGHT)
fig2.write_image(add_step3_path("residuals__{}.png".format(plt_seed)),
                width=FIG_WIDTH, height=FIG_HEIGHT)

### Model 6: Recursive ridge regression + BOCD

In [38]:
""" Model setup """

import numpy as np
from scipy import stats

# Parameters here
ridge_beta = np.zeros((REG_L, 1))
model_initialized = False
Qn_initialized = False
df_pred_vs_actual = []
residuals = np.zeros(x_obs.shape)

### BOCD 
bc = bocd.BayesianOnlineChangePointDetection(bocd.ConstantHazard(300), bocd.StudentT(mu=0, kappa=1, alpha=1, beta=1))
rt_mle = np.empty(x_obs.shape)

""" Train sequentially """
timerange = range(REG_L, x_obs.size-1) # data starts at 0 so start analysis at L for L lags
timerange = range(REG_L, REG_L + PROTOTYPE_TIME_STEPS)
x_postprocess = []
y_postprocess = []
for i in timerange:
    if (i % 10000 == 0): print("Training observation {} / {} ...".format(i+1, n-1))
    y = x_obs[i].tolist()[0]
    x = np.array(x_obs[(i-REG_L):i]).reshape(-1,)
    
    # impute mean if lags are NaN, if not possible then put NA and continue
    if np.isnan([x]).any():
        # impute mean if possible, if not then do NA
        x_nonna = x[~np.isnan(x)]
        if len(x_nonna) < 1:
            df_pred_vs_actual.append([i, y, np.nan])
            continue
        else:
            x_impute = np.copy(x)
            x_impute[np.isnan(x_impute)] = np.nanmean(x_impute)
            x = x_impute
    
    # skip training if output is NA
    if (np.isnan([y]).any()):
        continue
    
    if (i <= BURN_IN_PERIOD):
        x_postprocess.append(x)
        y_postprocess.append(y)
        continue
        
    ### initialize ridge
    if not model_initialized:
        X_init = np.vstack(x_postprocess)
        Y_init = np.vstack(y_postprocess)
        
        ### Fit ridge once have enough data
        from sklearn.linear_model import Ridge
        ridge_clf = Ridge(alpha=1)
        ridge_clf.fit(X_init, Y_init)
        ridge_beta = ridge_clf.coef_.reshape(ridge_beta.shape)
        model_initialized = True

    X_curr = np.vstack(x_postprocess)
    Y_curr = np.vstack(y_postprocess)
    
    ### Predict current data point
    yhat = x.dot(ridge_beta)
    if (np.isnan(yhat).any()):
        break
    df_pred_vs_actual.append([i, y, yhat.item()])
    
    ### Train model
    
    # (1/2) - calculate matrix inverse every time (inefficient computation, faster convergence)
#     Qn_inv = sc.linalg.inv(X_curr.T.dot(X_curr) + 1 * np.eye(REG_L))
    # (2/2) - update matrix inverse recursively (efficient computation, slower convergence)
    if not Qn_initialized:
        Qn_inv = sc.linalg.inv(X_curr.T.dot(X_curr) + 1 * np.eye(REG_L))
        Qn_initialized = True
    else:
        Qn_update_num = (Qn_inv.dot(x.reshape(-1, 1).dot(x.reshape(-1, 1).T)).dot(Qn_inv))
        Qn_update_den = (1 + x.reshape(-1,1).T.dot(Qn_inv).dot(x.reshape(-1,1))).item()
        Qn_inv = Qn_inv - (Qn_update_num / Qn_update_den)
    
    # exact - working! (needs longer burn-in period)
    coefQn = (Qn_inv/(1 + x.reshape(-1, 1).T.dot(Qn_inv).dot(x.reshape(-1, 1))))
    
    # approx - does not work well
#     coefQn = (1/(np.power(i, 0.5)) * np.ones(ridge_beta.shape)).T

    xresid = x.T.reshape(-1, 1).dot(y - x.dot(ridge_beta)).reshape(-1, 1)
    # should approximate
    ridge_beta = ridge_beta + coefQn.dot(xresid)
    
    # store residual of actual vs. predicted
    residuals[i] = y - yhat
    
    """ Change point detection """
    
    r = residuals
    
    bc.update(r[i])
    rt_mle[i] = bc.rt

""" Plot """
df_plt = pd.DataFrame(df_pred_vs_actual)
df_plt.columns = ['t', 'actual', 'pred']
df_plt['resid'] = df_plt['actual'] - df_plt['pred']
# add change points
index_changes = np.where(np.diff(rt_mle.flatten())<0)[0]
df_plt2 = pd.merge(df_plt, pd.DataFrame({"t": index_changes, "break": 1}), on="t", how="left").fillna(0)
fig = px.line(df_plt2, x="t", y=['actual', 'pred'], template="plotly_dark")
fig.add_scatter(x=df_plt2['t'], y=df_plt2['actual'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig.update_layout(title="Actual vs. predicted")
fig.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig)

fig2 = px.line(df_plt2, x="t", y=['resid'], template="plotly_dark")
fig2.add_scatter(x=df_plt2['t'], y=df_plt2['resid'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig2.update_layout(title="Residuals")
fig2.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig2)

### Store modeling results
df_plt2['mse'] = np.mean((df_plt2['actual'] - df_plt2['pred'])**2)
df_plt2['rmse'] = np.sqrt(np.mean((df_plt2['actual'] - df_plt2['pred'])**2))
df_plt2['mae'] = np.mean(np.abs(df_plt2['pred'] - df_plt2['actual']))
df_plt2['mape'] = np.mean(100 * np.abs( 1 - df_plt2['pred'] / df_plt2['actual'] ))
df_plt2_final = df_plt2.assign(model_no=6, reg_model="Recursive ridge regression", change_model="BOCD")
df_all_modeling_results.append(df_plt2_final)

### Save plots
plt_seed = '_'.join(map(str,
    df_plt2_final[['model_no', 'reg_model', 'change_model']].iloc[0,].values))
fig.write_image(add_step3_path("actual_vs_pred__{}.png".format(plt_seed)),
               width=FIG_WIDTH, height=FIG_HEIGHT)
fig2.write_image(add_step3_path("residuals__{}.png".format(plt_seed)),
                width=FIG_WIDTH, height=FIG_HEIGHT)

### Model 7: Gaussian process regression + CUSUM

Non-sequential: train via expanding window every GP_RETRAIN_N iterations.

In [39]:
""" Model setup """

import numpy as np
from scipy import stats

# Model
MINIBATCH_RETRAIN_N = 100
model_initialized = False
df_pred_vs_actual = []
residuals = np.zeros(x_obs.shape)

### CUSUM parameters
threshold = 15 # increase for more strict
drift = 0
gp, gn = np.zeros(x_obs.size), np.zeros(x_obs.size)
ta, tai, taf = np.array([[], [], []], dtype=int)
tap, tan = 0, 0
amp = np.array([])

""" Train sequentially """
timerange = range(REG_L, x_obs.size-1) # data starts at 0 so start analysis at L for L lags
timerange = range(REG_L, REG_L + PROTOTYPE_TIME_STEPS)
x_postprocess = []
y_postprocess = []
for i in timerange:
    if (i % 10000 == 0): print("Training observation {} / {} ...".format(i+1, n-1))
    y = x_obs[i].tolist()[0]
    x = np.array(x_obs[(i-REG_L):i]).reshape(-1,)
    
    # impute mean if lags are NaN, if not possible then put NA and continue
    if np.isnan([x]).any():
        # impute mean if possible, if not then do NA
        x_nonna = x[~np.isnan(x)]
        if len(x_nonna) < 1:
            df_pred_vs_actual.append([i, y, np.nan])
            continue
        else:
            x_impute = np.copy(x)
            x_impute[np.isnan(x_impute)] = np.nanmean(x_impute)
            x = x_impute
    
    # skip training if output is NA
    if (np.isnan([y]).any()):
        continue
    
    if (i <= BURN_IN_PERIOD):
        x_postprocess.append(x)
        y_postprocess.append(y)
        continue
        
    ### initialize ridge
    if not model_initialized:
        X_init = np.vstack(x_postprocess)
        Y_init = np.vstack(y_postprocess)
        
        ### Fit model once enough data
        from sklearn.gaussian_process.kernels import RBF, WhiteKernel
        from sklearn.gaussian_process import GaussianProcessRegressor
        kernel = 1.0 * RBF(length_scale=100.0, length_scale_bounds=(1e-2, 1e3)) \
            + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))
        gp_reg = GaussianProcessRegressor(kernel=kernel).fit(X_init, Y_init.reshape(-1,))
        
        model_initialized = True

    # full data
    X_curr = np.vstack(x_postprocess)
    Y_curr = np.vstack(y_postprocess)
    
    ### Predict current data point
    yhat = gp_reg.predict(x.reshape(-1,1).T)
#     yhat = x.dot(ridge_beta)
    if (np.isnan(yhat).any()):
        break
    df_pred_vs_actual.append([i, y, yhat.item()])
    
    ### Train model in minibatch every GP_RETRAIN_N iterations
    if (i % MINIBATCH_RETRAIN_N == 0):
        gp_reg = gp_reg.fit(X_curr, Y_curr.reshape(-1,))
    
    # store residual of actual vs. predicted
    residuals[i] = y - yhat
    
    """ CUSUM """

    # CUSUM on the (online) residuals
    # (only run if already have 1 or more residuals calculated)
    r = residuals
    if (r[i-1] == 0):
        continue
    
    # Auto-estimate drift to be (1/2)*expected change
    # aka mean of the last REG_LAG residuals.
    drift = np.mean(residuals[(i-REG_L):i]) / 2
    
    # Auto-estimate threshold
    
    s = np.array(r[i]) - np.array(r[i-1])
    gp[i] = gp[i-1] + s - drift  # cumulative sum for + change
    gn[i] = gn[i-1] - s - drift  # cumulative sum for - change
    if gp[i] < 0:
        gp[i], tap = 0, i
    if gn[i] < 0:
        gn[i], tan = 0, i
    if gp[i] > threshold or gn[i] > threshold:  # change detected!
        ta = np.append(ta, i)    # alarm index
        tai = np.append(tai, tap if gp[i] > threshold else tan)  # start
        gp[i], gn[i] = 0, 0      # reset alarm


""" Plot """
df_plt = pd.DataFrame(df_pred_vs_actual)
df_plt.columns = ['t', 'actual', 'pred']
df_plt['resid'] = df_plt['actual'] - df_plt['pred']
df_plt2 = pd.merge(df_plt, pd.DataFrame({"t": ta, "break": 1}), on="t", how="left").fillna(0)
fig = px.line(df_plt2, x="t", y=['actual', 'pred'], template="plotly_dark")
fig.add_scatter(x=df_plt2['t'], y=df_plt2['actual'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig.update_layout(title="Actual vs. predicted")
fig.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig)

fig2 = px.line(df_plt2, x="t", y=['resid'], template="plotly_dark")
fig2.add_scatter(x=df_plt2['t'], y=df_plt2['resid'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig2.update_layout(title="Residuals")
fig2.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig2)

### Store modeling results
df_plt2['mse'] = np.mean((df_plt2['actual'] - df_plt2['pred'])**2)
df_plt2['rmse'] = np.sqrt(np.mean((df_plt2['actual'] - df_plt2['pred'])**2))
df_plt2['mae'] = np.mean(np.abs(df_plt2['pred'] - df_plt2['actual']))
df_plt2['mape'] = np.mean(100 * np.abs( 1 - df_plt2['pred'] / df_plt2['actual'] ))
df_plt2_final = df_plt2.assign(model_no=7, reg_model="Gaussian process regression", change_model="CUSUM")
df_all_modeling_results.append(df_plt2_final)

### Save plots
plt_seed = '_'.join(map(str,
    df_plt2_final[['model_no', 'reg_model', 'change_model']].iloc[0,].values))
fig.write_image(add_step3_path("actual_vs_pred__{}.png".format(plt_seed)),
               width=FIG_WIDTH, height=FIG_HEIGHT)
fig2.write_image(add_step3_path("residuals__{}.png".format(plt_seed)),
                width=FIG_WIDTH, height=FIG_HEIGHT)

### Model 8: Gaussian process regression + BOCD

In [40]:
""" Model setup """

import numpy as np
from scipy import stats

# Model
MINIBATCH_RETRAIN_N = 100
model_initialized = False
df_pred_vs_actual = []
residuals = np.zeros(x_obs.shape)

### BOCD 
bc = bocd.BayesianOnlineChangePointDetection(bocd.ConstantHazard(300), bocd.StudentT(mu=0, kappa=1, alpha=1, beta=1))
rt_mle = np.empty(x_obs.shape)

""" Train sequentially """
timerange = range(REG_L, x_obs.size-1) # data starts at 0 so start analysis at L for L lags
timerange = range(REG_L, REG_L + PROTOTYPE_TIME_STEPS)
x_postprocess = []
y_postprocess = []
for i in timerange:
    if (i % 10000 == 0): print("Training observation {} / {} ...".format(i+1, n-1))
    y = x_obs[i].tolist()[0]
    x = np.array(x_obs[(i-REG_L):i]).reshape(-1,)
    
    # impute mean if lags are NaN, if not possible then put NA and continue
    if np.isnan([x]).any():
        # impute mean if possible, if not then do NA
        x_nonna = x[~np.isnan(x)]
        if len(x_nonna) < 1:
            df_pred_vs_actual.append([i, y, np.nan])
            continue
        else:
            x_impute = np.copy(x)
            x_impute[np.isnan(x_impute)] = np.nanmean(x_impute)
            x = x_impute
    
    # skip training if output is NA
    if (np.isnan([y]).any()):
        continue
    
    if (i <= BURN_IN_PERIOD):
        x_postprocess.append(x)
        y_postprocess.append(y)
        continue
        
    ### initialize ridge
    if not model_initialized:
        X_init = np.vstack(x_postprocess)
        Y_init = np.vstack(y_postprocess)
        
        ### Fit model once enough data
        from sklearn.gaussian_process.kernels import RBF, WhiteKernel
        from sklearn.gaussian_process import GaussianProcessRegressor
        kernel = 1.0 * RBF(length_scale=100.0, length_scale_bounds=(1e-2, 1e3)) \
            + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))
        gp_reg = GaussianProcessRegressor(kernel=kernel).fit(X_init, Y_init.reshape(-1,))
        
        model_initialized = True

    # full data
    X_curr = np.vstack(x_postprocess)
    Y_curr = np.vstack(y_postprocess)
    
    ### Predict current data point
    yhat = gp_reg.predict(x.reshape(-1,1).T)
#     yhat = x.dot(ridge_beta)
    if (np.isnan(yhat).any()):
        break
    df_pred_vs_actual.append([i, y, yhat.item()])
    
    ### Train model in minibatch every GP_RETRAIN_N iterations
    if (i % MINIBATCH_RETRAIN_N == 0):
        gp_reg = gp_reg.fit(X_curr, Y_curr.reshape(-1,))
    
    # store residual of actual vs. predicted
    residuals[i] = y - yhat

    """ Change point detection """
    
    r = residuals
    
    bc.update(r[i])
    rt_mle[i] = bc.rt

""" Plot """
df_plt = pd.DataFrame(df_pred_vs_actual)
df_plt.columns = ['t', 'actual', 'pred']
df_plt['resid'] = df_plt['actual'] - df_plt['pred']
# add change points
index_changes = np.where(np.diff(rt_mle.flatten())<0)[0]
df_plt2 = pd.merge(df_plt, pd.DataFrame({"t": index_changes, "break": 1}), on="t", how="left").fillna(0)
fig = px.line(df_plt2, x="t", y=['actual', 'pred'], template="plotly_dark")
fig.add_scatter(x=df_plt2['t'], y=df_plt2['actual'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig.update_layout(title="Actual vs. predicted")
fig.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig)

fig2 = px.line(df_plt2, x="t", y=['resid'], template="plotly_dark")
fig2.add_scatter(x=df_plt2['t'], y=df_plt2['resid'], mode="markers",
                marker_size=df_plt2['break']*6, # because binary
                marker_color=df_plt2['break'])
fig2.update_layout(title="Residuals")
fig2.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
display(fig2)

### Store modeling results
df_plt2['mse'] = np.mean((df_plt2['actual'] - df_plt2['pred'])**2)
df_plt2['rmse'] = np.sqrt(np.mean((df_plt2['actual'] - df_plt2['pred'])**2))
df_plt2['mae'] = np.mean(np.abs(df_plt2['pred'] - df_plt2['actual']))
df_plt2['mape'] = np.mean(100 * np.abs( 1 - df_plt2['pred'] / df_plt2['actual'] ))
df_plt2_final = df_plt2.assign(model_no=8, reg_model="Gaussian process regression", change_model="BOCD")
df_all_modeling_results.append(df_plt2_final)

### Save plots
plt_seed = '_'.join(map(str,
    df_plt2_final[['model_no', 'reg_model', 'change_model']].iloc[0,].values))
fig.write_image(add_step3_path("actual_vs_pred__{}.png".format(plt_seed)),
               width=FIG_WIDTH, height=FIG_HEIGHT)
fig2.write_image(add_step3_path("residuals__{}.png".format(plt_seed)),
                width=FIG_WIDTH, height=FIG_HEIGHT)

# Combine all modeling results

In [41]:
""" Actual predictions - overall idea """
df_full = pd.concat(df_all_modeling_results)

### Save
df_full.assign(analysis_key=ANALYSIS_KEY).to_csv(add_step3_path("model_predictions_long.csv"))

### Create wide version with predictions in columns - for display only
index_col = "t"
model_cols = ["model_no", "reg_model", "change_model"]
error_cols = ['mse', 'rmse', 'mape', 'mae']
value_cols = [c for c in df_full.columns if c not in [index_col] + model_cols + error_cols]
df_full_wide = df_full.pivot(index=index_col, columns=model_cols, values=value_cols)
display(df_full_wide.head())


actual                                                                                                                                                              pred                                                                                                                                                             resid                                                                                                                                                             break                                                                                                                                                    
model_no                 1             2           3           4                          5                          6                           7                           8             1             2           3           4                          5                          6                           7                           8             1             2           3           4                          5                          6                           7                           8             1             2           3           4                          5                          6                           7                           8
reg_model    Kalman Filter Kalman Filter Bayesian LR Bayesian LR Recursive ridge regression Recursive ridge regression Gaussian process regression Gaussian process regression Kalman Filter Kalman Filter Bayesian LR Bayesian LR Recursive ridge regression Recursive ridge regression Gaussian process regression Gaussian process regression Kalman Filter Kalman Filter Bayesian LR Bayesian LR Recursive ridge regression Recursive ridge regression Gaussian process regression Gaussian process regression Kalman Filter Kalman Filter Bayesian LR Bayesian LR Recursive ridge regression Recursive ridge regression Gaussian process regression Gaussian process regression
change_model         CUSUM          BOCD       CUSUM        BOCD                      CUSUM                       BOCD                       CUSUM                        BOCD         CUSUM          BOCD       CUSUM        BOCD                      CUSUM                       BOCD                       CUSUM                        BOCD         CUSUM          BOCD       CUSUM        BOCD                      CUSUM                       BOCD                       CUSUM                        BOCD         CUSUM          BOCD       CUSUM        BOCD                      CUSUM                       BOCD                       CUSUM                        BOCD
t                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
20                  68.714        68.714         NaN         NaN                        NaN                        NaN                         NaN                         NaN        68.166        68.166         NaN         NaN                        NaN                        NaN                         NaN                         NaN         0.549         0.549         NaN         NaN                        NaN                        NaN                         NaN                         NaN         0.000         0.000         NaN         NaN                        NaN                        NaN                         NaN                         NaN
21                  69.000        69.000         NaN  

In [42]:
""" Summary error statistics """

# note: only for prediction part so will be same between change point detection models

df_summary_err = df_full.groupby(['model_no', 'reg_model', 'change_model']).agg({
    "mse": [len, np.mean],
    "rmse": np.mean,
    "mae": np.mean,
    "mape": np.mean})

### Save
df_summary_err.assign(analysis_key=ANALYSIS_KEY).to_csv(add_step3_path("summary_error.csv"))

# different burn-in times, etc.
display(df_summary_err)

mse            rmse    mae   mape
                                                      len     mean   mean   mean   mean
model_no reg_model                   change_model                                      
1        Kalman Filter               CUSUM        446.000   56.720  7.531  5.769  9.437
2        Kalman Filter               BOCD         446.000   56.720  7.531  5.769  9.437
3        Bayesian LR                 CUSUM        410.000   47.010  6.856  4.923  8.005
4        Bayesian LR                 BOCD         410.000   47.010  6.856  4.923  8.005
5        Recursive ridge regression  CUSUM        410.000 2002.369 44.748 33.366 49.967
6        Recursive ridge regression  BOCD         410.000 2002.369 44.748 33.366 49.967
7        Gaussian process regression CUSUM        410.000   59.801  7.733  5.697  9.672
8        Gaussian process regression BOCD         410.000   59.801  7.733  5.697  9.672

In [43]:
""" Summary error statistics - cumulative """

cum_grps = []
for key, grp in df_full.groupby(['model_no', 'reg_model', 'change_model']):
    df_grp = grp.reset_index()
    df_grp = df_grp.assign(resid_sqr=lambda x: (x.actual - x.pred)**2,
              resid_abs=lambda x: np.abs(x.actual - x.pred),
              resid_abspct=lambda x: 100*np.abs(1 - (x.pred / x.actual)))
    
    # Cumulative MSE, RMSE, MAE, MAPE
    df_grp['cum_mse'] = df_grp['resid_sqr'].expanding(1).mean()
    df_grp['cum_rmse'] = df_grp['resid_sqr'].expanding(1).agg({"rmse": lambda x: np.sqrt(np.mean(x))})
    df_grp['cum_mae'] = df_grp['resid_abs'].expanding(1).mean()
    df_grp['cum_mape'] = df_grp['resid_abspct'].expanding(1).mean()
    df_grp = df_grp.drop(columns=['resid_sqr', 'resid_abs', 'resid_abspct'])
                           
    cum_grps.append(df_grp)

df_summary_cum_err = pd.concat(cum_grps)
df_summary_cum_err['model_key'] = df_summary_cum_err.reg_model + " + " + df_summary_cum_err.change_model
display(df_summary_cum_err.head())

### Save
df_summary_cum_err.assign(analysis_key=ANALYSIS_KEY).to_csv(add_step3_path("summary_error_cumulative.csv"))

""" Plot errs over time """

err_metrics = ['cum_mse', 'cum_rmse', 'cum_mae', 'cum_mape']
for err_metric in err_metrics:
    plt_title = ' '.join(["Model {}:".format(v) if isinstance(v, int)
          else str(v) for v in
          df_summary_cum_err[['model_no', 'reg_model', 'change_model']].head(1).values.flatten()])
    plt_seed = '_'.join(["model{}".format(v) if isinstance(v, int)
          else str(v) for v in
          df_summary_cum_err[['model_no', 'reg_model', 'change_model']].head(1).values.flatten()])
    
    ### Plot figure
    df_plt_tmp = df_summary_cum_err[['model_key', 't', err_metric]]
    fig = px.line(df_plt_tmp,
                  x="t", y=err_metric, template="plotly_dark", line_group="model_key", color="model_key")
    fig.update_layout(title="Sequential {} over time".format(err_metric.replace("_", " ").title()), legend_title="")
    fig.update_layout(xaxis_title="Time [seconds]", yaxis_title=err_metric.replace("_", " ").title())
    display(fig)
    fig.write_image(add_step3_path("sequential_{}.png".format(err_metric)),
                width=FIG_WIDTH, height=FIG_HEIGHT)

,index,t,actual,pred,resid,break,mse,rmse,mae,mape,model_no,reg_model,change_model,cum_mse,cum_rmse,cum_mae,cum_mape,model_key
0,0,20,68.714,68.166,0.549,0.000,56.720,7.531,5.769,9.437,1,Kalman Filter,CUSUM,0.301,0.549,0.549,0.798,Kalman Filter + CUSUM
1,1,21,69.000,68.164,0.836,0.000,56.720,7.531,5.769,9.437,1,Kalman Filter,CUSUM,0.500,0.707,0.692,1.005,Kalman Filter + CUSUM
2,2,22,63.250,68.051,-4.801,0.000,56.720,7.531,5.769,9.437,1,Kalman Filter,CUSUM,8.018,2.832,2.062,3.200,Kalman Filter + CUSUM
3,3,23,69.000,67.664,1.336,0.000,56.720,7.531,5.769,9.437,1,Kalman Filter,CUSUM,6.460,2.542,1.881,2.884,Kalman Filter + CUSUM
4,4,24,71.750,67.639,4.111,0.000,56.720,7.531,5.769,9.437,1,Kalman Filter,CUSUM,8.548,2.924,2.327,3.453,Kalman Filter + CUSUM


# Compare structural breaks against manual tagging

Focus on how many was able to get (+/- before/after break) and compare relative ability of methods.

In [44]:
if (core_filenames[0] not in core_filenames_legend or
    TIME_SERIES_VARIABLE not in core_filenames_legend[core_filenames[0]]):
    raise Exception("Cannot run structural break validation (no manual tagging.)")

Exception: Cannot run structural break validation (no manual tagging.)

In [ ]:
""" Read in my manually tagged structural breaks """

# assume only 1 core music stream midi file for now (aka pick the first file)
breaks_tagged_midi = pretty_midi.PrettyMIDI(core_filenames_legend[core_filenames[0]][TIME_SERIES_VARIABLE])
df_breaks_raw = pd.DataFrame([(n.start, n.end, n.pitch, n.velocity, n.duration)
                          for n in breaks_tagged_midi.instruments[0].notes],
                         columns=['start', 'end', 'pitch', 'velocity', 'duration'])

# Note, not totally a fair comparison because of the burn-in period for sequential learning.
# Report anyway.

# +/- margin for manually tagged structural break
STRUCTURAL_BREAK_MARGIN_GRID = np.arange(0.10, 1, 0.05)
df_precision_recall_list = []
for margin in STRUCTURAL_BREAK_MARGIN_GRID:
    print("Running for margin {} ...".format(margin))

    # use MIDI note (start +/- STRUCTURAL_BREAK_MARGIN [seconds]) as the structural break range
    df_breaks = df_breaks_raw.copy()
    df_breaks['break_onset'] = (df_breaks['start'] - margin).apply(lambda x: max(x, 0))
    df_breaks['break_end'] = (df_breaks['start'] + margin).apply(
        lambda x: min(x, df_maestro_agg.start_sec.max()+1))
    df_breaks = df_breaks[['break_onset', "start", 'break_end']].sort_values(by="start")
    df_breaks.columns = ['break_onset', 'point', 'break_end']
    df_breaks.head()

    """ Create subdataframe of breaks detected by the models (to nearest second). """

    ### Overall precision and recall
    # Precision: (breaks correctly predicted) / (all breaks predicted)
    # = (true positives) / (true positives + false positives = predicted positives)
    df_breaks_detected = df_full.loc[df_full['break'] > 0, ['t', 'break', 'model_no', 'reg_model', 'change_model']]
    df_breaks_detected['detected'] = df_breaks_detected['t'].apply(
        lambda t: max(df_breaks.eval('(break_onset <= {}) & ({} <= break_end)'.format(t, t))))
    N_PRED_POS = df_breaks_detected.shape[0] # predicted
    N_TRUE_POS = df_breaks_detected['detected'].sum() # correctly predicted
    precision = N_TRUE_POS / N_PRED_POS
#     print("Overall precision:")
#     print(precision)
    # Recall: (breaks correctly predicted) / (all relevant time steps to consider)
    # = (true positives) / (true positives + false negatives = correctly predictive positives + positives we missed)
    # Here, calculate false negative # (FNN) - number of manually tagged structural breaks we did not catch
    df_breaks['failed_to_detect'] = df_breaks.apply(
        lambda x: max(df_breaks_detected.eval('({} <= t) & (t <= {})'.format(
            float(x[['break_onset']]), float(x[['break_end']])))), axis=1)
    N_FAILED_TO_DETECT = df_breaks['failed_to_detect'].sum() # false negatives
    recall = N_TRUE_POS / (N_TRUE_POS + N_FAILED_TO_DETECT)
#     print("Overall recall:")
#     print(recall)
    f1 = 2 * (precision * recall) / (precision + recall)
    df_summary = pd.DataFrame({
        "margin": margin, 
        "model_no": "all", "pred_model": "all", "change_model": "all",
        "precision": precision, "recall": recall, "f1": f1}, index=[0])
    df_precision_recall_list.append(df_summary)

    """ Do by breakdown now """
    for key, grp in df_breaks_detected.groupby(['model_no', 'reg_model', 'change_model']):
#         print(key)

        ### Overall precision and recall
        # Precision: (breaks correctly predicted) / (all breaks predicted)
        # = (true positives) / (true positives + false positives = predicted positives)
        grp = df_full.loc[df_full['break'] > 0, ['t', 'break', 'model_no', 'reg_model', 'change_model']]
        grp = grp.loc[grp['model_no'] == key[0],]
        grp['detected'] = grp['t'].apply(
            lambda t: max(df_breaks.eval('(break_onset <= {}) & ({} <= break_end)'.format(t, t))))
        N_PRED_POS = grp.shape[0] # predicted
        N_TRUE_POS = grp['detected'].sum() # correctly predicted
        precision = N_TRUE_POS / N_PRED_POS
#         print("Overall precision:")
#         print(precision)
        # Recall: (breaks correctly predicted) / (all relevant time steps to consider)
        # = (true positives) / (true positives + false negatives = correctly predictive positives + positives we missed)
        # Here, calculate false negative # (FNN) - number of manually tagged structural breaks we did not catch
        df_breaks['failed_to_detect'] = df_breaks.apply(
            lambda x: max(grp.eval('({} <= t) & (t <= {})'.format(
                float(x[['break_onset']]), float(x[['break_end']])))), axis=1)
        N_FAILED_TO_DETECT = df_breaks['failed_to_detect'].sum() # false negatives
        recall = N_TRUE_POS / (N_TRUE_POS + N_FAILED_TO_DETECT)
#         print("Overall recall:")
#         print(recall)
        f1 = 2 * (precision * recall) / (precision + recall)
        df_summary = pd.DataFrame({
            "margin": margin, 
            "model_no": key[0], "pred_model": key[1], "change_model": key[2],
            "precision": precision, "recall": recall, "f1": f1}, index=[0])
        df_precision_recall_list.append(df_summary)

""" Compile, display and save results """

df_precision_recall = pd.concat(df_precision_recall_list)
display(df_precision_recall)
df_precision_recall.to_csv(add_step3_path("structural_breaks_precision_recall.csv"))

In [ ]:
# Note, can get NAN for very tough margins if denominator is zero for either precision or recall.
display(df_precision_recall)

In [ ]:
""" Lastly: plot my manually tagged structural breaks vs. model predictions on the time series. """

df_ts = df_maestro_agg[['start_sec', TIME_SERIES_VARIABLE]] # confirmed same as ts
df_breaks_manual = df_breaks_raw[['start']].copy().reset_index() # original tagged times of structural breaks
df_breaks_manual = df_breaks_manual.loc[df_breaks_manual['index'] >= df_ts.start_sec.min(),]
df_breaks_manual = df_breaks_manual.loc[df_breaks_manual['index'] <= df_ts.start_sec.max(),]
df_breaks_manual.columns = ['start_sec', 'break']
df_breaks_manual['ypos'] = 20
df_breaks_pred = df_full.loc[df_full['break'] > 0, ["actual", 't', 'model_no', 'reg_model', 'change_model']]

display(df_ts.head())
display(df_breaks_manual.head())
df_breaks_pred.head()

# Range to show in plot
timerange = range(df_ts.start_sec.min(), df_ts.start_sec.max(), 1)
fig_breaks = px.line(df_ts.loc[timerange,], x="start_sec", y=TIME_SERIES_VARIABLE, template="plotly_dark")
fig_breaks.update_layout(xaxis_title="Time [seconds]", yaxis_title=TIME_SERIES_VARIABLE_FMT)
fig_breaks.add_scatter(x=df_breaks_manual.loc[df_breaks_manual['break'] <= max(timerange), 'break'],
                       y=df_breaks_manual.loc[df_breaks_manual['break'] <= max(timerange), 'ypos'],
                       mode="markers",
                marker_size=10, # because binary
                marker_color="white", marker_symbol="star", name="Manually tagged")

colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
markers = ["circle", "square", "diamond", "cross", "x", "triangle-up", "pentagon", "hexagram"]
for mx, (key, grp) in enumerate(df_breaks_pred.groupby(['model_no', 'reg_model', 'change_model'])):
    df_curr = grp[['t', 'actual']]
    df_curr = df_curr.loc[df_curr['t'] <= max(timerange),]
    df_curr['ypos'] = 20 + 2*(mx+1)
    fig_breaks.add_scatter(x=df_curr.t, y=df_curr.ypos, mode="markers", marker_size=10,
                          marker_color=colors[mx], marker_symbol=markers[mx],
                           name=key[1] + "-" + key[2])

### Plot
fig_breaks.update_layout(title="Structural breaks [x-axis] in {}: manually tagged vs. detected".format(
    TIME_SERIES_VARIABLE_FMT))
display(fig_breaks)
fig_breaks.write_image(add_step3_path("structural_breaks_plot.png"),
            width=FIG_WIDTH*np.floor(df_ts.start_sec.max() / 50), height=FIG_HEIGHT)